In [557]:
!pip install PyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1FMYnVj7N02DBBxFqykIUEY_p01BY79fE'})

In [0]:
download.GetContentFile('sign_mnist_train.csv')

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import pydot

In [0]:
train = pd.read_csv('sign_mnist_train.csv', header=0)

In [0]:
download = drive.CreateFile({'id': '1jNwbta8bjZlG28Tf9nMHRToPG5bNkZt3'})

In [0]:
download.GetContentFile('sign_mnist_test.csv')

In [0]:
test = pd.read_csv('sign_mnist_test.csv', header=0)

In [0]:
y_train = train['label'].values
y_test = test['label'].values

In [0]:
# We have grayscale images, so while loading the images we will keep grayscale=True, if you have RGB images, you should set grayscale as False
X_train = train.drop(['label'],axis=1)
X_test = test.drop(['label'], axis=1)

X_train = np.array(X_train.iloc[:,:])
X_train = np.array([np.reshape(i, (28,28)) for i in X_train])

X_test = np.array(X_test.iloc[:,:])
X_test = np.array([np.reshape(i, (28,28)) for i in X_test])

#for i in tqdm(range(train.shape[0])):
#    img = image.load_img('train/'+ train[i].astype('str')+'.png', target_size=(28,28,1), grayscale=False)
#    img = image.img_to_array(image)
#    img = img/255
#    train_image.append(img)
#X = np.array(train_image)

In [0]:
num_classes = 26
y_train = np.array(y_train).reshape(-1)
y_test = np.array(y_test).reshape(-1)

y_train = np.eye(num_classes)[y_train]
y_test = np.eye(num_classes)[y_test]

In [570]:
print('Training examples:',X_train.shape[0])
print('Test examples:',X_test.shape[0])

print('X_train shape"',X_train.shape)
print('y_train shape"',y_train.shape)
print('X_test shape"',X_test.shape)
print('y_test shape"',y_test.shape)

Training examples: 27455
Test examples: 7172
X_train shape" (27455, 28, 28)
y_train shape" (27455, 26)
X_test shape" (7172, 28, 28)
y_test shape" (7172, 26)


In [0]:

X_train = X_train.reshape((27455, 28, 28, 1))
X_test = X_test.reshape((7172, 28, 28, 1))

In [0]:
def model():
  classifier = Sequential()
  classifier.add(Convolution2D(filters=8, kernel_size=(3,3),strides=(1,1),padding='same',input_shape=(28,28,1),activation='relu', data_format='channels_last'))
  classifier.add(MaxPooling2D(pool_size=(2,2)))
  classifier.add(Convolution2D(filters=16, kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'))
  classifier.add(MaxPooling2D(pool_size=(4,4)))
  classifier.add(Flatten())
  classifier.add(Dense(128, activation='relu'))
  classifier.add(Dense(26, activation='softmax'))
  classifier.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
  return classifier

In [0]:
classifier = model()

In [574]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_101 (Conv2D)          (None, 28, 28, 8)         80        
_________________________________________________________________
max_pooling2d_101 (MaxPoolin (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_102 (Conv2D)          (None, 14, 14, 16)        1168      
_________________________________________________________________
max_pooling2d_102 (MaxPoolin (None, 3, 3, 16)          0         
_________________________________________________________________
flatten_51 (Flatten)         (None, 144)               0         
_________________________________________________________________
dense_100 (Dense)            (None, 128)               18560     
_________________________________________________________________
dense_101 (Dense)            (None, 26)                3354      
Total para

In [575]:
classifier.fit(X_train, y_train, batch_size = 100, epochs = 100)

Epoch 1/100
27455/27455 [==============================] - 14s 519us/step - loss: 1.9975 - acc: 0.4757
Epoch 2/100
27455/27455 [==============================] - 11s 418us/step - loss: 0.3775 - acc: 0.8758
Epoch 3/100
27455/27455 [==============================] - 12s 420us/step - loss: 0.1143 - acc: 0.9653
Epoch 4/100
27455/27455 [==============================] - 11s 418us/step - loss: 0.0600 - acc: 0.9832
Epoch 5/100
27455/27455 [==============================] - 12s 431us/step - loss: 0.0413 - acc: 0.9895
Epoch 6/100
27455/27455 [==============================] - 11s 419us/step - loss: 0.0300 - acc: 0.9918
Epoch 7/100
27455/27455 [==============================] - 11s 418us/step - loss: 0.0297 - acc: 0.9921
Epoch 8/100
27455/27455 [==============================] - 12s 421us/step - loss: 0.0207 - acc: 0.9940
Epoch 9/100
27455/27455 [==============================] - 11s 418us/step - loss: 0.0156 - acc: 0.9957
Epoch 10/100
27455/27455 [==============================] - 12s 423us/ste

In [0]:
y_pred = classifier.predict(X_test)

In [577]:
y_pred = np.argmax(y_pred, axis=1)
y_pred

array([ 6,  5, 10, ...,  2,  4,  2])

In [578]:
y_test = np.array(test['label'].values)
y_test

array([ 6,  5, 10, ...,  2,  4,  2])

In [579]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8796709425543782